In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
import os
 
dirpath = os.getcwd()
print("current directory is : " + dirpath)

current directory is : /content


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# textgeneration.txt
# /content/gdrive/My Drive/DL/Project/textgeneration.txt

text=(open("/content/gdrive/My Drive/DL/Project/shakespear.txt").read())
text=text.lower()

In [0]:
characters = sorted(list(set(text)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [0]:
X = []
Y = []
length = len(text)
seq_length = 100

for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [0]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [8]:
X_modified, Y_modified

(array([[[0.5       ],
         [0.        ],
         [0.        ],
         ...,
         [0.27777778],
         [0.77777778],
         [0.02777778]],
 
        [[0.        ],
         [0.        ],
         [0.02777778],
         ...,
         [0.77777778],
         [0.02777778],
         [0.80555556]],
 
        [[0.        ],
         [0.02777778],
         [0.41666667],
         ...,
         [0.02777778],
         [0.80555556],
         [0.47222222]],
 
        ...,
 
        [[0.36111111],
         [0.02777778],
         [0.30555556],
         ...,
         [0.77777778],
         [0.02777778],
         [0.38888889]],
 
        [[0.02777778],
         [0.30555556],
         [0.38888889],
         ...,
         [0.02777778],
         [0.38888889],
         [0.63888889]],
 
        [[0.30555556],
         [0.38888889],
         [0.41666667],
         ...,
         [0.38888889],
         [0.63888889],
         [0.36111111]]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 

In [9]:
model = Sequential()
model.add(LSTM(600, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(600))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 600)          1444800   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 600)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 600)               2882400   
_________________________________________________________________
dropout_2 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 36)                21636     
Total params: 4,348,836
Trainable params: 4,348,836
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(X_modified, Y_modified, epochs=1, batch_size=50)

Epoch 1/1
19188/19188 [==============================] - 249s 13ms/step - loss: 3.0271


In [0]:
#model 1
model.save_weights('/content/gdrive/My Drive/DL/Project/new_poem_weights/wider_model_1_epoch')

In [0]:
model.load_weights('/content/gdrive/My Drive/DL/Project/new_poem_weights/wider_model_1_epoch')

In [0]:
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [14]:
#combining text
txt=""
for char in full_string:
    txt = txt+char
txt

' the riper should by time decease,\n his tender heir might bear his memory:\n but thou, contracted to                                                                                                                                                                                                                                                                                                                                                                                                                 '

In [15]:
'''
" the riper should by time decease,\n his tender heir might bear his memory:\n 
but thou, contracted to thine own bright eyes,\n were ana lek sreechls of thine
age shalt see,\n de tty fr the soart dod that mure ro he.\n   to eat the world's
due, by the grave and thee.\n\n 

ii\n\n when forty winters shall besiege thy brow
,\n and dig deep trenches in thy beauty's field,\n thy youth's proud livery so 
gazed on now,\n will be a tat mnte anmver thae, le has aliodsing,\n haply i
think on thee,-- and then my state,\n li"
'''

'\n" the riper should by time decease,\n his tender heir might bear his memory:\n \nbut thou, contracted to thine own bright eyes,\n were ana lek sreechls of thine\nage shalt see,\n de tty fr the soart dod that mure ro he.\n   to eat the world\'s\ndue, by the grave and thee.\n\n \n\nii\n\n when forty winters shall besiege thy brow\n,\n and dig deep trenches in thy beauty\'s field,\n thy youth\'s proud livery so \ngazed on now,\n will be a tat mnte anmver thae, le has aliodsing,\n haply i\nthink on thee,-- and then my state,\n li"\n'

In [0]:
'''
'\n" the riper should by time decease,\n his tender heir might bear his memory:
\n \nbut thou, contracted to thine own bright eyes,\n were ana lek sreechls 
of thine\nage shalt see,\n de tty fr the soart dod that mure ro he.\n  
to eat the world\'s\ndue, by the grave and thee.\n\n \n\


nii\n\n when forty winters shall besiege thy brow\n,\n and dig deep trenche
s in thy beauty\'s field,\n thy youth\'s proud livery so \ngazed on now,\n
will be a tat mnte anmver thae, le has aliodsing,\n haply i\nthink on thee,--
and then my state,\n li"\n'

'''